In [1]:
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from VideoSwinTransformer import *
import os
from collections import OrderedDict

In [2]:
torch.set_grad_enabled(False)

In [3]:
def get_x(shape= (1,3,32,224,224)):
    x_pt = torch.rand(shape) * 255
    x_np = x_pt.numpy()
    x_tf = tf.convert_to_tensor(x_np)

    return x_tf, x_pt

In [4]:
shape_of_input = [2,3,32,224,224]
x_tf, x_pt = get_x(shape_of_input)

Metal device set to: Apple M1

systemMemory: 8.00 GB


2022-09-03 23:54:31.446521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-03 23:54:31.446631: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


maxCacheSize: 2.67 GB



## Convert Weights

Initiate model and Load PyTorch Weights

In [5]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']
del cfg["drop_path_rate"]
# download_weight_command = f"wget {link} -O {name}.pth"
# os.system(download_weight_command)

In [6]:
shape_of_input[1] , shape_of_input[4], shape_of_input[2] = shape_of_input[2], shape_of_input[1], shape_of_input[4]

# (2, 32, 224,224,3)
# shape_of_input = [1,3,32,224,224]

print("shape_of_input", shape_of_input)
print("pt")
pt_model = SwinTransformer3D_pt(**cfg,drop_rate=0.4, drop_path_rate=0., isTest= True)
print("++++++++\n tf")
tf_model = SwinTransformer3D(**cfg,shape_of_input=shape_of_input, drop_rate=0.4, drop_path_rate=0., isTest= True)
x_tf, x_pt = get_x()


print("\n\n\n pt")
basic_pt, z= pt_model(x_pt)
print("------- \n tf")

basic_tf, y = tf_model(x_tf)

print("\n loading checkpoint")
checkpoint = torch.load(f'{name}.pth')
new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    if 'backbone' in k:
        name = k[9:]
        new_state_dict[name] = v 

pt_model.load_state_dict(new_state_dict) 
pt_model.eval()

shape_of_input [2, 32, 224, 224, 3]
pt
++++++++
 tf
shape_of_input:  [2, 32, 224, 224, 3]


/Users/mohammadshoaib/Codes/tensorflow-test/env/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]





 pt
------- 
 tf

 loading checkpoint


In [7]:
# a = a

In [8]:
pt_model.eval()


Convert Functions

In [9]:
def conv_transpose(w):
    return w.transpose(2,3,4,1, 0)
    

def modify_tf_block( tf_component, pt_weight,  pt_bias = None, is_attn=False):
    in_shape = pt_weight.shape

    if isinstance(tf_component, tf.keras.layers.Conv3D) :
      pt_weight = conv_transpose(pt_weight)

    if isinstance(tf_component, tf.keras.layers.Dense) and not is_attn:
      pt_weight =pt_weight.transpose()

    if isinstance(tf_component, (tf.keras.layers.Dense, tf.keras.layers.Conv3D)):
        tf_component.kernel.assign(tf.Variable(pt_weight))

        if pt_bias is not None:
            tf_component.bias.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, tf.keras.layers.LayerNormalization):

        tf_component.gamma.assign(tf.Variable(pt_weight))

        tf_component.beta.assign(tf.Variable(pt_bias))

    elif isinstance(tf_component, (tf.Variable)):
        tf_component.assign(tf.Variable(pt_weight))

    else:
        return tf.convert_to_tensor(pt_weight)
        
        

    return tf_component


def modify_swin_blocks(np_state_dict, pt_weights_prefix, tf_block):

  for layer in tf_block:
    if isinstance(layer, PatchMerging):
      patch_merging_idx = f"{pt_weights_prefix}.downsample"

      layer.reduction = modify_tf_block( layer.reduction,
                          np_state_dict[f"{patch_merging_idx}.reduction.weight"])
      layer.norm = modify_tf_block( layer.norm,
                        np_state_dict[f"{patch_merging_idx}.norm.weight"],
                        np_state_dict[f"{patch_merging_idx}.norm.bias"]
                        )
      
  # Swin Layers
  common_prefix = f"{pt_weights_prefix}.blocks"
  block_idx = 0

  for outer_layer in tf_block:

      layernorm_idx = 1
      mlp_layer_idx = 1

      if isinstance(outer_layer, SwinTransformerBlock3D):
          for inner_layer in outer_layer.layers:
        
              # Layer norm.
              if isinstance(inner_layer, tf.keras.layers.LayerNormalization):
                  layer_norm_prefix = (
                      f"{common_prefix}.{block_idx}.norm{layernorm_idx}"
                  )
                  inner_layer.gamma.assign(
                      tf.Variable(
                          np_state_dict[f"{layer_norm_prefix}.weight"]
                      )
                  )



                  inner_layer.beta.assign(
                      tf.Variable(np_state_dict[f"{layer_norm_prefix}.bias"])
                  )

                  layernorm_idx += 1

              # Window attention.
              elif isinstance(inner_layer, WindowAttention3D):
                  attn_prefix = f"{common_prefix}.{block_idx}.attn"

                  # Relative position.
                  inner_layer.relative_position_bias_table = (
                      modify_tf_block(
                          inner_layer.relative_position_bias_table,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_bias_table"
                          ] 
                      )
                  )
                  inner_layer.relative_position_index = (
                      modify_tf_block(
                          inner_layer.relative_position_index,
                          np_state_dict[
                              f"{attn_prefix}.relative_position_index"
                          ]
                      )
                  )

                  # QKV.
                  inner_layer.qkv = modify_tf_block(
                      inner_layer.qkv,
                      np_state_dict[f"{attn_prefix}.qkv.weight"],
                      np_state_dict[f"{attn_prefix}.qkv.bias"]
                  )

                  # Projection.
                  inner_layer.proj = modify_tf_block(
                      inner_layer.proj,
                      np_state_dict[f"{attn_prefix}.proj.weight"],
                      np_state_dict[f"{attn_prefix}.proj.bias"]
                  )

              # MLP.
              elif isinstance(inner_layer, tf.keras.Model):
                  mlp_prefix = f"{common_prefix}.{block_idx}.mlp"
                  for mlp_layer in inner_layer.layers:
                      if isinstance(mlp_layer, tf.keras.layers.Dense):
                          mlp_layer = modify_tf_block(
                              mlp_layer,
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.weight"
                              ],
                              np_state_dict[
                                  f"{mlp_prefix}.fc{mlp_layer_idx}.bias"
                              ]
                          )
                          mlp_layer_idx += 1

          block_idx += 1
  return tf_block


### convert

In [10]:
np_state_dict = pt_model.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}

tf_model.projection.layers[0] = modify_tf_block(tf_model.projection.layers[0]
        ,
        np_state_dict["patch_embed.proj.weight"],
        np_state_dict["patch_embed.proj.bias"])

tf_model.projection.layers[1] = modify_tf_block(
    tf_model.projection.layers[1],
    np_state_dict["patch_embed.norm.weight"],
    np_state_dict["patch_embed.norm.bias"])


layer_normalization_idx = -1

tf_model.layers[layer_normalization_idx] = modify_tf_block(
    tf_model.layers[layer_normalization_idx] ,
    np_state_dict["norm.weight"],
    np_state_dict["norm.bias"]
    )

# swin layers
for i in range(2, len(tf_model.layers) - 1):
    _ = modify_swin_blocks(np_state_dict,
                        f"layers.{i-2}",
                        tf_model.layers[i].layers)

In [11]:
x_tf, x_pt = get_x()

In [12]:
layers_output_tf, y = tf_model(x_tf)
print("------")
layers_output_pt, z= pt_model(x_pt)

y.shape, z.shape

# basic 1 input (1, 96, 4, 56, 56)

------


(TensorShape([1, 768, 16, 7, 7]), torch.Size([1, 768, 16, 7, 7]))

In [13]:
# Eplore the output of all layers
for layer in layers_output_pt:
    print("--------------",layer, "-------------\n TF: ", layers_output_tf[layer].numpy()[:1,:1,:1,:1,:10], "\n PT: ", layers_output_pt[layer].detach().numpy()[:1,:1,:1,:1,:10], "\n" )

-------------- PatchEmbed -------------
 TF:  [[[[[-0.20840067  0.03822488 -0.7638208  -3.4460974  -0.13888127
     -0.06092447  0.8322891  -1.1731992  -1.5243905  -1.4762405 ]]]]] 
 PT:  [[[[[-0.20840046  0.03822553 -0.7638214  -3.4460983  -0.13888127
     -0.06092435  0.83228946 -1.1731994  -1.5243908  -1.4762404 ]]]]] 

-------------- drop_out -------------
 TF:  [[[[[-0.20840067  0.03822488 -0.7638208  -3.4460974  -0.13888127
     -0.06092447  0.8322891  -1.1731992  -1.5243905  -1.4762405 ]]]]] 
 PT:  [[[[[-0.20840046  0.03822553 -0.7638214  -3.4460983  -0.13888127
     -0.06092435  0.83228946 -1.1731994  -1.5243908  -1.4762404 ]]]]] 

-------------- basic layer1 -------------
 TF:  [[[[[-0.3186322   0.5461706   0.04582031  0.645411    0.53313243
      0.08724829  0.76752603  0.76062554  0.0594265   0.8415976 ]]]]] 
 PT:  [[[[[-0.31863272  0.5461707   0.04582013  0.6454112   0.53313214
      0.08724847  0.767526    0.76062536  0.05942651  0.8415983 ]]]]] 

-------------- basic laye

In [14]:
# compare layers' output. It asserts  at basic layer1
for layer in layers_output_pt:

    abs_diff = np.sum( np.abs( layers_output_tf[layer].numpy() - layers_output_pt[layer].detach().numpy()))

    print(f"{abs_diff} Absolute Difference of  {layer}" )


1.1764469146728516 Absolute Difference of  PatchEmbed
1.1764469146728516 Absolute Difference of  drop_out
0.5647180080413818 Absolute Difference of  basic layer1
0.3812922239303589 Absolute Difference of  basic layer2
0.4626581072807312 Absolute Difference of  basic layer3
1.2589079141616821 Absolute Difference of  basic layer4
0.32200807332992554 Absolute Difference of  Final Output


In [26]:
# compare layers' output. It asserts  at basic layer1
for layer in layers_output_pt:
    print("Testing", layer)
    np.testing.assert_allclose(layers_output_tf[layer].numpy(), layers_output_pt[layer].detach().numpy(), 1e-4, 1e-4)

Testing PatchEmbed
Testing drop_out
Testing basic layer1
Testing basic layer2
Testing basic layer3
Testing basic layer4
Testing Final Output


In [16]:
e = 1e-4
np.testing.assert_allclose(layers_output_tf["basic layer4"].numpy(), layers_output_pt["basic layer4"].detach().numpy(), e, e)


In [17]:
# comparing the outputs
np.testing.assert_allclose(y.numpy(), z.detach().numpy(), 1e-4, 1e-4)


### PT basic layer outputs comparison

In [18]:
attempts = 2
# x_pt = torch.rand((1,3,8,224,224))

outputs = []

for i in range(attempts):
    layer_out , result = pt_model(x_pt)
    outputs.append(layer_out)

i = 0
for layer in outputs[0]:
    print("--------------", layer, "---------------")
    for idx, layer_out in enumerate(outputs) :
        print(f"attempt {idx} : ",layer_out[layer].detach().numpy()[:1,:1,:1,:1,:10], "\n")
    print()


-------------- PatchEmbed ---------------
attempt 0 :  [[[[[-0.20840046  0.03822553 -0.7638214  -3.4460983  -0.13888127
     -0.06092435  0.83228946 -1.1731994  -1.5243908  -1.4762404 ]]]]] 

attempt 1 :  [[[[[-0.20840046  0.03822553 -0.7638214  -3.4460983  -0.13888127
     -0.06092435  0.83228946 -1.1731994  -1.5243908  -1.4762404 ]]]]] 


-------------- drop_out ---------------
attempt 0 :  [[[[[-0.20840046  0.03822553 -0.7638214  -3.4460983  -0.13888127
     -0.06092435  0.83228946 -1.1731994  -1.5243908  -1.4762404 ]]]]] 

attempt 1 :  [[[[[-0.20840046  0.03822553 -0.7638214  -3.4460983  -0.13888127
     -0.06092435  0.83228946 -1.1731994  -1.5243908  -1.4762404 ]]]]] 


-------------- basic layer1 ---------------
attempt 0 :  [[[[[-0.31863272  0.5461707   0.04582013  0.6454112   0.53313214
      0.08724847  0.767526    0.76062536  0.05942651  0.8415983 ]]]]] 

attempt 1 :  [[[[[-0.31863272  0.5461707   0.04582013  0.6454112   0.53313214
      0.08724847  0.767526    0.76062536  0.

### All close testing

Compare the first attempt with another attempt. Enter the attempt value in the following cell

In [19]:
attempt_no = 1     # Change the attempt_no value to compare the first attempt with another attempt


In [20]:
# PatchEmbed Layer
output1 = outputs[0]["PatchEmbed"]
output2 =  outputs[attempt_no]["PatchEmbed"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [21]:
# Basic Layer1
output1 = outputs[0]["basic layer1"]
output2 =  outputs[attempt_no]["basic layer1"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [22]:
# Basic Layer2

output1 = outputs[0]["basic layer2"]
output2 =  outputs[attempt_no]["basic layer2"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [23]:
# Basic Layer3

output1 = outputs[0]["basic layer3"]
output2 =  outputs[attempt_no]["basic layer3"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [24]:
# Basic Layer4

output1 = outputs[0]["basic layer4"]
output2 =  outputs[attempt_no]["basic layer4"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)

In [25]:
# Final Output

output1 = outputs[0]["Final Output"]
output2 =  outputs[attempt_no]["Final Output"]

np.testing.assert_allclose(output1.detach().numpy(), output2.detach().numpy(), 1e-4, 1e-4)